In [3]:
import pandas as pd
import numpy as np
import os
import time
import subprocess
import re
import random
import arff

from math import sqrt
from sklearn.model_selection import  train_test_split
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB  
from sklearn.metrics import classification_report
from sklearn.metrics import coverage_error
from sklearn.metrics import label_ranking_loss
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import zero_one_loss
from sklearn.metrics import jaccard_similarity_score

# functions

In [6]:
def read_data(dataPath):
    # input: '/Volumes/Samsung_T5/research/data/ABC_news_data/obesity/'
    # read data
    data = pd.read_csv(os.path.join(dataPath,'X.csv'))
    data.drop(columns='protein',inplace=True)
    data.replace({'YES':1, 'NO':0},inplace=True)
    label = pd.read_csv(os.path.join(dataPath,'Y.csv'))
    return data,label

def evaluation(y_pred, y_prob, y_true):
    
    coverage = coverage_error(y_true, y_prob)
    hamming = hamming_loss(y_true, y_pred)
    ranking_loss = label_ranking_loss(y_true, y_prob) 
    
    f1_macro = metrics.f1_score(y_true, y_pred, average='macro')
    f1_micro = metrics.f1_score(y_true, y_pred, average='micro')
    
    acc = 0
    for i in range(y_true.shape[0]):
        acc += jaccard_similarity_score(y_true.iloc[i,:], y_pred.iloc[i,:]) # jaccard_similarity_score
    acc = round(acc / y_true.shape[0],2)
    
    zero_one = zero_one_loss(y_true, y_pred) # 0-1 error 
    
    f1_each = metrics.f1_score(y_true, y_pred, average=None)
    
    performance = {"coverage_error":coverage,
                   "ranking_loss":ranking_loss,
                   "hamming_loss":hamming,
                   "f1_macro":f1_macro,
                   "f1_micro":f1_micro,
                   "Jaccard_Index":acc,
                   "zero_one_error":zero_one,
                   "f1_each_label":f1_each}
    return performance

def get_confusion_matrix(y_pred, y_test, column_names):
    """confusion matrix """
    confusion_matrix = pd.DataFrame(np.array(y_pred) - np.array(y_test), columns=column_names)
    pos = pd.DataFrame((np.array(y_pred) == np.array(y_test)) & (np.array(y_pred) == 1), columns=y_test.columns).sum(axis=0)
    neg = pd.DataFrame((np.array(y_pred) == np.array(y_test)) & (np.array(y_pred) == 0), columns=y_test.columns).sum(axis=0)
    for i in range(confusion_matrix.shape[1]): 
        name = confusion_matrix.iloc[:,i].name
        temp = confusion_matrix.iloc[:,i].value_counts()
        TP = pos[name]
        TN = neg[name]
        if 1 in temp.index:
            FP = temp[1]
        else:
            FP = 0
        if -1 in temp.index:
            FN = temp[-1]
        else:
            FN = 0

# read data



In [5]:
dataPath = '/Volumes/Samsung_T5/research/data/small_datasets/genbase/'
dataset = 'genbase'
data, label = read_data(dataPath) # read data

# get data information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()
avg_instance_per_label = label.sum(axis=0).mean()
# print data information
print("\n--- Data Information ---")
print("dataset:", dataPath)
print("number of label:",n_label)
print("number of attribute:",n_attr)
print("number of instance:",n_instance)
print("avgerage number of labels for an instance:",avg_label_per_instance)
print("avgerage number of positive instances for a label:",avg_instance_per_label,"the std:",sqrt(label.sum(axis=0).var()),"\n")

print("-- number of positive instances --")
print(label.sum(axis=0))


--- Data Information ---
dataset: /Volumes/Samsung_T5/research/data/small_datasets/genbase/
number of label: 27
number of attribute: 1185
number of instance: 662
avgerage number of labels for an instance: 1.2522658610271904
avgerage number of positive instances for a label: 30.703703703703702 the std: 37.35053785506546 

-- number of positive instances --
PDOC00154     79
PDOC00343     76
PDOC00271     62
PDOC00064     49
PDOC00791    171
PDOC00380     23
PDOC50007     31
PDOC00224     51
PDOC00100      6
PDOC00670     66
PDOC50002     33
PDOC50106     29
PDOC00561     36
PDOC50017     14
PDOC50003     14
PDOC50006      4
PDOC50156     17
PDOC00662     41
PDOC00018      9
PDOC50001      5
PDOC00014      2
PDOC00750      3
PDOC50196      2
PDOC50199      1
PDOC00660      1
PDOC00653      1
PDOC00030      3
dtype: int64


In [33]:
cooccurrence_matrix = label.T.dot(label)
np.fill_diagonal(cooccurrence_matrix.values, 0)
#cooccurrence_matrix.to_csv('/Users/jiangjunhao/Desktop/cooccurrence_matrix.csv', index=False)
cooccurrence_matrix

,PDOC00154,PDOC00343,PDOC00271,PDOC00064,PDOC00791,PDOC00380,PDOC50007,PDOC00224,PDOC00100,PDOC00670,...,PDOC00662,PDOC00018,PDOC50001,PDOC00014,PDOC00750,PDOC50196,PDOC50199,PDOC00660,PDOC00653,PDOC00030
PDOC00154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,2,1,0,0,3
PDOC00343,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PDOC00271,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,0,0,1,1,0
PDOC00064,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PDOC00791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
PDOC00380,0,0,0,0,0,0,23,0,0,0,...,0,9,5,0,0,0,0,0,0,0
PDOC50007,0,0,0,0,0,23,0,0,0,0,...,0,9,5,0,0,0,0,0,0,0
PDOC00224,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PDOC00100,0,0,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
PDOC00670,0,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0


# Binary Relevance 

## BR using naive Bayes classifier

## BR using ESKDB

In [34]:
def get_arff(word_occurrence, label_matrix, savePath): # get attributes
    for z in range(len(label_matrix.columns)):
        attributes=[(word_occurrence.columns[i],list(map(str,sorted(word_occurrence.iloc[:,i].unique())))) for i in range(len(word_occurrence.columns))]
        attributes.append(('label_'+label_matrix.columns[z],['0', '1']))

        data=[]
        i = 0
        while i < label_matrix.shape[0]:
            attr_data = [str(j) for j in list(word_occurrence.iloc[i,:])]
            label_data = [str(label_matrix.iloc[i,z])]
            row_data = attr_data+label_data
            data.append(row_data) 
            i+=1
        # set obj
        obj = {
           'description': u'',
           'relation': 'relation',
           'attributes': attributes,
           'data': data,
        }
        arff_data = arff.dumps(obj)
        w_file = open(savePath+label_matrix.columns[z]+".arff", "w")
        w_file.write(arff_data)
        w_file.close()

def run_eskdb(dataPath, resultFile, k, l, e, i):
    command = "./run_eskdb.sh "+resultFile+" "+k+" "+i+" "+l+" "+e+" "+dataPath
    subprocess.call("cd /Volumes/Samsung_T5/research/programme/research_python/", shell=True)
    print(command)
    return subprocess.call(command, shell=True)

def get_result(resultPath):

    y_pred = pd.DataFrame()
    y_true = pd.DataFrame()
    y_prob = pd.DataFrame()
    names = []
    for file in os.listdir(resultPath):
        with open(os.path.join(resultPath,file), 'r') as f:
            try:
                lines = f.readlines()
            except:
                print(file)
            else:
                names.append(file[:-4])
                pred = []
                true = []
                prob = []
                train_time_total = 0
                test_time_total = 0
                error_marco = 0
                for line in lines:
                    if line.startswith('pred'):
                        pred.append(int(re.search('pred :\t(.)',line).group(1)))
                        true.append(int(re.search('true :\t(.)',line).group(1)))
                        prob.append(float(re.search('prob :\t(.*)',line).group(1)))
                    elif line.startswith('RSME'):
                        rsme = float(re.search('RSME :\t\t(.*)',line).group(1))
                    elif line.startswith('Error'):
                        error = float(re.search('Error :\t\t(.*)',line).group(1))
                    elif line.startswith("Training time"):
                        train_time = float(re.search('Training time :\s{1,}(.*)',line).group(1))
                        train_time_total = train_time_total + train_time
                    elif line.startswith("Testing time"):
                        test_time = float(re.search('Testing time :\s{1,}(.*)',line).group(1))
                        test_time_total = test_time_total + test_time
                    elif line.startswith("["):
                        para = line
                    elif line.startswith("test0Indexes"):
                        index = list(map(int,re.search('test0Indexes: {(.*)}',line).group(1).split(', ')))

                y_pred = pd.concat([y_pred,pd.DataFrame(pred)],axis=1)
                y_true = pd.concat([y_true,pd.DataFrame(true)],axis=1)
                y_prob = pd.concat([y_prob,pd.DataFrame(prob)],axis=1)
    y_pred.columns = names
    y_true.columns = names
    y_pred.index = index
    y_true.index = index
    print(para)
    print("number of label:", y_pred.shape[1])
    print("training time:",train_time_total)
    print("testing time:",test_time_total)
    return y_pred,y_true,y_prob,index

In [35]:
# read data
data, label = read_data(dataPath)

# get arff files

savePath = "/Users/jiangjunhao/Desktop/test/"+dataset+'/'
if not os.path.exists(savePath):
    os.mkdir(savePath)
get_arff(data,label,savePath)

resultFile = dataset+'_k5_e20_i5000'
k = '5'
i = '5000'
l = '2'
e = '20'

run_eskdb(savePath, resultFile, k, l, e, i)

./run_eskdb.sh genbase_k5_e20_i5000 5 5000 2 20 /Users/jiangjunhao/Desktop/test/genbase/


0

In [36]:
resultPath = '/Volumes/Samsung_T5/research/programme/ESKDB_HDP/result/'+resultFile
y_pred,y_true,y_prob,index = get_result(resultPath)

performance = evaluation(y_pred=y_pred, y_true=y_true, y_prob=y_prob)
    
# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

[-t, /Users/jiangjunhao/Desktop/test/genbase//PDOC50199.arff, -S, ESKDB, -K, 5, -I, 5000, -L, 2, -E, 20, -V, -M]

number of label: 27
training time: 5134.0
testing time: 393.0
--- Performance ---
coverage_error = 1.18 ( avg_label_per_instance = 1.25 )
ranking_loss = 0.0
hamming_loss = 0.0
f1_macro = 0.77
f1_micro = 0.99
Jaccard_Index = 1.0
zero_one_error = 0.02

- f1 for each label -
label_PDOC00154 = 0.0
label_PDOC00343 = 1.0
label_PDOC00271 = 1.0
label_PDOC00064 = 1.0
label_PDOC00791 = 0.0
label_PDOC00380 = 1.0
label_PDOC50007 = 1.0
label_PDOC00224 = 1.0
label_PDOC00100 = 1.0
label_PDOC00670 = 1.0
label_PDOC50002 = 1.0
label_PDOC50106 = 0.0
label_PDOC00561 = 0.0
label_PDOC50017 = 1.0
label_PDOC50003 = 1.0
label_PDOC50006 = 1.0
label_PDOC50156 = 1.0
label_PDOC00662 = 1.0
label_PDOC00018 = 0.96
label_PDOC50001 = 1.0
label_PDOC00014 = 1.0
label_PDOC00750 = 0.73
label_PDOC50196 = 1.0
label_PDOC50199 = 1.0
label_PDOC00660 = 1.0
label_PDOC00653 = 0.0
label_PDOC00030 = 0.0


/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [37]:
def naiveBayes_multi_label_training(X_train, y_train):
    start = time.time()
    
    n_label = y_train.shape[1]
    classifier_list = [MultinomialNB() for i in range(n_label)]
    for i in range(n_label):
        classifier_list[i].fit(X_train,y_train.iloc[:,i])
    
    end = time.time()
    training_time = end-start
    
    return classifier_list, training_time

def naiveBayes_multi_label_testing(X_test, n_label, classifier_list):
    y_predict = pd.DataFrame()
    y_prob = pd.DataFrame()
    
    start = time.time()
    
    for i in range(n_label):
        y_predict_i = classifier_list[i].predict(X_test)
        y_predict = pd.concat([y_predict, pd.DataFrame(y_predict_i)],axis=1)
        
        y_predict_prob_i = classifier_list[i].predict_proba(X_test)[:,1]
        y_prob = pd.concat([y_prob, pd.DataFrame(y_predict_prob_i)],axis=1)
        
    end = time.time()
    testing_time = end-start
        
    return y_predict, y_prob, testing_time

def BR_test(data, label, dataPath, random_state=3071980):
    
    # data set information
    n_label = label.shape[1]
    n_attr = data.shape[1]
    n_instance = data.shape[0]
    avg_label_per_instance = label.sum(axis=1).mean()
    
    # split training and test data set
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.5, random_state=random_state)
    
    print("-- test index --")
    print(X_test.index)
    
    # training
    classifier_list, training_time = naiveBayes_multi_label_training(X_train, y_train)
    
    # testing
    y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list)
    
    # evaluation
    performance = evaluation(y_predict, y_prob, y_test)
    
    # print data information
    print("--- Data Information ---")
    print("dataset:", dataPath)
    print("number of label:",n_label)
    print("number of attribute:",n_attr)
    print("number of instance:",n_instance,"\n")
    
    # get confusion matrix
    get_confusion_matrix(y_predict, y_test, y_test.columns)
    
    # print performance
    print("--- Performance ---")
    for key, value in performance.items():
        if key == "f1_each_label":
            print("\n- f1 for each label -")
            for i in range(n_label):
                print("label_"+label.columns[i],"=",round(value[i],2))
        elif key == "coverage_error":
            print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
        else:
            print(key,'=',round(value,2))
            
# two fold cross-validation
def two_fold_BR_test(data, label, dataPath, n_iter=5, random_state=3071980):
    
    # data set information
    n_label = label.shape[1]
    n_attr = data.shape[1]
    n_instance = data.shape[0]
    avg_label_per_instance = label.sum(axis=1).mean()
    
    # 2-fold cross validatiom
    KF=KFold(n_splits=2, shuffle=True, random_state=random_state)
    i = 0
    
    performance = {}
    for train_index,test_index in KF.split(data):
        i += 1
        
        X_train,X_test=data.iloc[train_index,:],data.iloc[test_index,:]
        y_train,y_test=label.iloc[train_index,:],label.iloc[test_index,:]
        
        print("--- kfold time="+str(i)+" ---")
        # training
        classifier_list, training_time = naiveBayes_multi_label_training(X_train, y_train)
        # testing
        y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list)
        
        # evaluation
        if performance == {}:
            performance = evaluation(y_predict, y_prob, y_test)
        else:
            performance_i = evaluation(y_predict, y_prob, y_test)
            for key, value in performance_i.items():
                performance[key] = (performance[key] + value)/2
            else:
                performance[key] = value
    
    # print data information
    print("\n--- Data Information ---")
    print("dataset:", dataPath)
    print("number of label:",n_label)
    print("number of attribute:",n_attr)
    print("number of instance:",n_instance,"\n")
    
    # print performance
    print("--- 2 fold cross-validation Performance ---")
    for key, value in performance.items():
        if key == "f1_each_label":
            continue
        elif key == "coverage_error":
            print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
        else:
            print(key,'=',round(value,2))

# read data
data, label = read_data(dataPath)

# train - test
print("------ Binary Relevance using Naive Bayes ------")
BR_test(data, label, dataPath,3071980)

#print("")
#print("------ two_fold Binary Relevance using Naive Bayes ------")
#two_fold_BR_test(data, label, dataPath,3071980)

# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

# split training and test data set
# X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.5, random_state=random_state)
X_train = data[data.index.isin(index)==False]
X_test = data[data.index.isin(index)==True]
y_train = label[label.index.isin(index)==False]
y_test = label[label.index.isin(index)==True]

print("-- test index --")
print(X_test.index)

# training
print("--- start training ---\n")
classifier_list, training_time = naiveBayes_multi_label_training(X_train, y_train)

# testing
print("--- start testing ---\n")
y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list)

# get confusion matrix
get_confusion_matrix(y_predict, y_test, y_test.columns)
    
# evaluation
performance = evaluation(y_predict, y_prob, y_test)


# print data information
print("--- Data Information ---")
print("dataset:", dataPath)
print("number of label:",n_label)
print("number of attribute:",n_attr)
print("number of instance:",n_instance,"\n")

# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

------ Binary Relevance using Naive Bayes ------
-- test index --
Int64Index([597,  91, 171,  42,  88, 180, 580, 332, 214, 257,
            ...
            261, 632, 505, 618, 487,  33, 502, 222, 647, 375],
           dtype='int64', length=331)


/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


--- Data Information ---
dataset: /Volumes/Samsung_T5/research/data/small_datasets/genbase/
number of label: 27
number of attribute: 1185
number of instance: 662 

--- Performance ---
coverage_error = 1.37 ( avg_label_per_instance = 1.25 )
ranking_loss = 0.0
hamming_loss = 0.01
f1_macro = 0.48
f1_micro = 0.93
Jaccard_Index = 0.99
zero_one_error = 0.13

- f1 for each label -
label_PDOC00154 = 0.96
label_PDOC00343 = 0.98
label_PDOC00271 = 0.98
label_PDOC00064 = 0.98
label_PDOC00791 = 0.99
label_PDOC00380 = 0.86
label_PDOC50007 = 0.86
label_PDOC00224 = 1.0
label_PDOC00100 = 0.0
label_PDOC00670 = 1.0
label_PDOC50002 = 0.92
label_PDOC50106 = 1.0
label_PDOC00561 = 1.0
label_PDOC50017 = 0.44
label_PDOC50003 = 0.0
label_PDOC50006 = 0.0
label_PDOC50156 = 0.0
label_PDOC00662 = 1.0
label_PDOC00018 = 0.0
label_PDOC50001 = 0.0
label_PDOC00014 = 0.0
label_PDOC00750 = 0.0
label_PDOC50196 = 0.0
label_PDOC50199 = 0.0
label_PDOC00660 = 0.0
label_PDOC00653 = 0.0
label_PDOC00030 = 0.0
-- test index --
Int

/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


--- Data Information ---
dataset: /Volumes/Samsung_T5/research/data/small_datasets/genbase/
number of label: 27
number of attribute: 1185
number of instance: 662 

--- Performance ---
coverage_error = 1.27 ( avg_label_per_instance = 1.25 )
ranking_loss = 0.0
hamming_loss = 0.0
f1_macro = 0.55
f1_micro = 0.94
Jaccard_Index = 1.0
zero_one_error = 0.11

- f1 for each label -
label_PDOC00154 = 0.92
label_PDOC00343 = 1.0
label_PDOC00271 = 0.97
label_PDOC00064 = 1.0
label_PDOC00791 = 0.98
label_PDOC00380 = 0.86
label_PDOC50007 = 0.78
label_PDOC00224 = 1.0
label_PDOC00100 = 0.0
label_PDOC00670 = 1.0
label_PDOC50002 = 0.91
label_PDOC50106 = 0.95
label_PDOC00561 = 1.0
label_PDOC50017 = 0.67
label_PDOC50003 = 0.4
label_PDOC50006 = 0.0
label_PDOC50156 = 0.83
label_PDOC00662 = 0.85
label_PDOC00018 = 0.67
label_PDOC50001 = 0.0
label_PDOC00014 = 0.0
label_PDOC00750 = 0.0
label_PDOC50196 = 0.0
label_PDOC50199 = 0.0
label_PDOC00660 = 0.0
label_PDOC00653 = 0.0
label_PDOC00030 = 0.0


# Classifier Chain

## CC using naive Bayes

In [38]:
def naiveBayes_multi_label_training(X_train, y_train):
    start = time.time()
    
    n_label = y_train.shape[1]
    
    order = random.sample(list(range(n_label)),n_label) # get orders
    
    classifier_list = [MultinomialNB() for i in range(n_label)] # create a classifier chain
    
    for i in range(n_label):
        if i == 0:
            classifier_list[i].fit(X_train,y_train.iloc[:, order[i]])
        else:
            X_train = pd.concat([X_train, y_train.iloc[:,order[i-1]]],axis=1) # put the previous label into attribute space
            classifier_list[i].fit(X_train,y_train.iloc[:,order[i]])

    end = time.time()
    training_time = end-start
    
    return classifier_list, training_time, order

def naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order):
    y_predict = pd.DataFrame(index=X_test.index)
    y_prob = pd.DataFrame(index=X_test.index)
    y_true = pd.DataFrame(index=X_test.index)
    
    start = time.time()
    
    for i in range(n_label):
        y_predict_i = classifier_list[i].predict(X_test)
        y_predict = pd.concat([y_predict, pd.DataFrame(y_predict_i,index=X_test.index)],axis=1)

        y_predict_prob_i = classifier_list[i].predict_proba(X_test)[:,1]
        y_prob = pd.concat([y_prob, pd.DataFrame(y_predict_prob_i,index=X_test.index)],axis=1)

        X_test = pd.concat([X_test, pd.DataFrame(y_predict_i,index=X_test.index)],axis=1,ignore_index=True) # put the previous label into attribute space

    end = time.time()
    testing_time = end-start
        
    return y_predict, y_prob, testing_time

def CC_test(data, label, dataPath, random_state=3071980):
    
    # data set information
    n_label = label.shape[1]
    n_attr = data.shape[1]
    n_instance = data.shape[0]
    avg_label_per_instance = label.sum(axis=1).mean()
    
    # split training and test data set
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=random_state)
    
    # training
    print("--- start training ---\n")
    classifier_list, training_time, order = naiveBayes_multi_label_training(X_train, y_train)
    
    # testing
    print("--- start testing ---\n")
    y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order)
    
    # evaluation
    y_test = y_test.iloc[:,order]
    performance = evaluation(y_predict, y_prob, y_test)
    
    # print data information
    print("--- Data Information ---")
    print("dataset:", dataPath)
    print("number of label:",n_label)
    print("number of attribute:",n_attr)
    print("number of instance:",n_instance,"\n")
    
    # print orders
    print("\n--- Order of the chain ---")
    print(label.columns[order])
    print("")
    
    # print performance
    print("--- Performance ---")
    for key, value in performance.items():
        if key == "f1_each_label":
            print("\n- f1 for each label -")
            for i in range(n_label):
                print("label_"+label.columns[i],"=",round(value[i],2))
        elif key == "coverage_error":
            print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
        else:
            print(key,'=',round(value,2))
            
def ECC_test(data, label, dataPath, random_state=3071980, ensemble = 5):
    
    # data set information
    n_label = label.shape[1]
    n_attr = data.shape[1]
    n_instance = data.shape[0]
    avg_label_per_instance = label.sum(axis=1).mean()
    
    # split training and test data set
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.5, random_state=random_state)
    
    # ensemble
    y_pred_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
    y_prob_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
    for i in range(ensemble):
        # training
        #print("--- start training ---\n")
        classifier_list, training_time, order = naiveBayes_multi_label_training(X_train, y_train)

        # print orders
        print("Order of the chain:",label.columns[order])

        # testing
        #print("--- start testing ---\n")
        y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order)

        y_predict.columns = label.columns[order]
        y_prob.columns = label.columns[order]
        y_predict = y_predict[label.columns]
        y_prob = y_prob[label.columns]

        y_pred_ensemble = y_pred_ensemble + y_predict
        y_prob_ensemble = y_prob_ensemble + y_prob

    y_pred_ensemble = (((y_pred_ensemble / ensemble) >= 0.5)*1).astype('int')
    y_prob_ensemble = y_prob_ensemble / ensemble 
    
    # evaluation
    performance = evaluation(y_pred_ensemble, y_prob_ensemble, y_test)
    
    # print data information
    print("--- Data Information ---")
    print("dataset:", dataPath)
    print("number of label:",n_label)
    print("number of attribute:",n_attr)
    print("number of instance:",n_instance,"\n")
    
    # print orders
    print("\n--- Order of the chain ---")
    print(label.columns[order])
    print("")
    
    # print performance
    print("--- Performance ---")
    for key, value in performance.items():
        if key == "f1_each_label":
            print("\n- f1 for each label -")
            for i in range(n_label):
                print("label_"+label.columns[i],"=",round(value[i],2))
        elif key == "coverage_error":
            print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
        else:
            print(key,'=',round(value,2))

In [39]:
# read data
data, label = read_data(dataPath)

# train - test
print("------ Binary Relevance using Naive Bayes ------")
ECC_test(data, label, dataPath, 3071980, 1)

------ Binary Relevance using Naive Bayes ------
Order of the chain: Index(['PDOC00100', 'PDOC50007', 'PDOC50003', 'PDOC50199', 'PDOC00653',
       'PDOC00064', 'PDOC00014', 'PDOC50196', 'PDOC00343', 'PDOC00670',
       'PDOC00561', 'PDOC00271', 'PDOC00750', 'PDOC00380', 'PDOC50002',
       'PDOC00662', 'PDOC00018', 'PDOC50001', 'PDOC00030', 'PDOC00791',
       'PDOC50006', 'PDOC50106', 'PDOC00224', 'PDOC00660', 'PDOC00154',
       'PDOC50017', 'PDOC50156'],
      dtype='object')
--- Data Information ---
dataset: /Volumes/Samsung_T5/research/data/small_datasets/genbase/
number of label: 27
number of attribute: 1185
number of instance: 662 


--- Order of the chain ---
Index(['PDOC00100', 'PDOC50007', 'PDOC50003', 'PDOC50199', 'PDOC00653',
       'PDOC00064', 'PDOC00014', 'PDOC50196', 'PDOC00343', 'PDOC00670',
       'PDOC00561', 'PDOC00271', 'PDOC00750', 'PDOC00380', 'PDOC50002',
       'PDOC00662', 'PDOC00018', 'PDOC50001', 'PDOC00030', 'PDOC00791',
       'PDOC50006', 'PDOC50106', 'P

/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [40]:
# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

# split training and test data set
# X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.5, random_state=random_state)

X_train = data[data.index.isin(index)==False]
X_test = data[data.index.isin(index)==True]
y_train = label[label.index.isin(index)==False]
y_test = label[label.index.isin(index)==True]

print("-- test index --")
print(X_test.index)

# training
print("--- start training ---\n")
classifier_list, training_time, order = naiveBayes_multi_label_training(X_train, y_train)

# testing
print("--- start testing ---\n")
y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order)

# evaluation
y_test = y_test.iloc[:,order]
performance = evaluation(y_predict, y_prob, y_test)

# print data information
print("--- Data Information ---")
print("dataset:", dataPath)
print("number of label:",n_label)
print("number of attribute:",n_attr)
print("number of instance:",n_instance,"\n")

# print orders
print("\n--- Order of the chain ---")
print(label.columns[order])
print("")


# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

-- test index --
Int64Index([  5,   7,   8,  15,  16,  17,  21,  23,  24,  31,
            ...
            648, 650, 651, 652, 655, 656, 657, 658, 660, 661],
           dtype='int64', length=331)
--- start training ---

--- start testing ---



/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


--- Data Information ---
dataset: /Volumes/Samsung_T5/research/data/small_datasets/genbase/
number of label: 27
number of attribute: 1185
number of instance: 662 


--- Order of the chain ---
Index(['PDOC50106', 'PDOC00100', 'PDOC00660', 'PDOC00154', 'PDOC50003',
       'PDOC00343', 'PDOC00791', 'PDOC50007', 'PDOC00561', 'PDOC00224',
       'PDOC50001', 'PDOC50199', 'PDOC00018', 'PDOC00750', 'PDOC50017',
       'PDOC00030', 'PDOC00271', 'PDOC50156', 'PDOC00380', 'PDOC00064',
       'PDOC50196', 'PDOC50006', 'PDOC00653', 'PDOC00014', 'PDOC00670',
       'PDOC00662', 'PDOC50002'],
      dtype='object')

--- Performance ---
coverage_error = 1.26 ( avg_label_per_instance = 1.25 )
ranking_loss = 0.0
hamming_loss = 0.0
f1_macro = 0.57
f1_micro = 0.95
Jaccard_Index = 1.0
zero_one_error = 0.1

- f1 for each label -
label_PDOC00154 = 0.95
label_PDOC00343 = 0.0
label_PDOC00271 = 0.0
label_PDOC00064 = 0.92
label_PDOC00791 = 0.4
label_PDOC00380 = 1.0
label_PDOC50007 = 0.98
label_PDOC00224 = 0.78
l

## Ensemble CC using naive Bayes (E = 10)

In [41]:
# read data
data, label = read_data(dataPath)

# train - test
print("------ Binary Relevance using Naive Bayes ------")
ECC_test(data, label, dataPath, random_state=3071980, ensemble=10)

------ Binary Relevance using Naive Bayes ------
Order of the chain: Index(['PDOC00750', 'PDOC00670', 'PDOC00100', 'PDOC00014', 'PDOC00660',
       'PDOC50017', 'PDOC00791', 'PDOC00064', 'PDOC50156', 'PDOC50001',
       'PDOC00271', 'PDOC00343', 'PDOC00380', 'PDOC00662', 'PDOC50106',
       'PDOC00561', 'PDOC50003', 'PDOC00154', 'PDOC00030', 'PDOC50006',
       'PDOC50196', 'PDOC00018', 'PDOC50002', 'PDOC50199', 'PDOC00653',
       'PDOC00224', 'PDOC50007'],
      dtype='object')
Order of the chain: Index(['PDOC50001', 'PDOC00100', 'PDOC50006', 'PDOC50156', 'PDOC00380',
       'PDOC50199', 'PDOC00670', 'PDOC00154', 'PDOC00014', 'PDOC00271',
       'PDOC00660', 'PDOC50106', 'PDOC00750', 'PDOC00791', 'PDOC00224',
       'PDOC50002', 'PDOC50017', 'PDOC00662', 'PDOC00018', 'PDOC00030',
       'PDOC00064', 'PDOC00653', 'PDOC50196', 'PDOC50007', 'PDOC00343',
       'PDOC00561', 'PDOC50003'],
      dtype='object')
Order of the chain: Index(['PDOC00271', 'PDOC00224', 'PDOC00343', 'PDOC00653', 

/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [42]:
# read data
data, label = read_data(dataPath)

ensemble=10

# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

X_train = data[data.index.isin(index)==False]
X_test = data[data.index.isin(index)==True]
y_train = label[label.index.isin(index)==False]
y_test = label[label.index.isin(index)==True]

print("-- test index --")
print(X_test.index)

# ensemble
y_pred_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
y_prob_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
for i in range(ensemble):
    # training
    #print("--- start training ---\n")
    classifier_list, training_time, order = naiveBayes_multi_label_training(X_train, y_train)

    # print orders
    print("Order of the chain:",label.columns[order])

    # testing
    #print("--- start testing ---\n")
    y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order)

    y_predict.columns = label.columns[order]
    y_prob.columns = label.columns[order]
    y_predict = y_predict[label.columns]
    y_prob = y_prob[label.columns]

    y_pred_ensemble = y_pred_ensemble + y_predict
    y_prob_ensemble = y_prob_ensemble + y_prob

y_pred_ensemble = (((y_pred_ensemble / ensemble) >= 0.5)*1).astype('int')
y_prob_ensemble = y_prob_ensemble / ensemble 

# evaluation
performance = evaluation(y_pred_ensemble, y_prob_ensemble, y_test)

# print data information
print("--- Data Information ---")
print("dataset:", dataPath)
print("number of label:",n_label)
print("number of attribute:",n_attr)
print("number of instance:",n_instance,"\n")

# print orders
print("\n--- Order of the chain ---")
print(label.columns[order])
print("")

# get confusion matrix
get_confusion_matrix(y_pred_ensemble, y_test, y_test.columns)

# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

-- test index --
Int64Index([  5,   7,   8,  15,  16,  17,  21,  23,  24,  31,
            ...
            648, 650, 651, 652, 655, 656, 657, 658, 660, 661],
           dtype='int64', length=331)
Order of the chain: Index(['PDOC00750', 'PDOC50001', 'PDOC00653', 'PDOC00380', 'PDOC50156',
       'PDOC00791', 'PDOC50007', 'PDOC50002', 'PDOC00224', 'PDOC50199',
       'PDOC00660', 'PDOC00030', 'PDOC50017', 'PDOC50196', 'PDOC00018',
       'PDOC00100', 'PDOC00662', 'PDOC00064', 'PDOC00271', 'PDOC50006',
       'PDOC00343', 'PDOC00561', 'PDOC00670', 'PDOC50003', 'PDOC00014',
       'PDOC50106', 'PDOC00154'],
      dtype='object')


/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


Order of the chain: Index(['PDOC50156', 'PDOC00750', 'PDOC50002', 'PDOC00271', 'PDOC00014',
       'PDOC50196', 'PDOC00064', 'PDOC50007', 'PDOC00343', 'PDOC00224',
       'PDOC00561', 'PDOC00791', 'PDOC00660', 'PDOC00662', 'PDOC50003',
       'PDOC00018', 'PDOC00030', 'PDOC50001', 'PDOC50106', 'PDOC00653',
       'PDOC00380', 'PDOC50199', 'PDOC00154', 'PDOC50017', 'PDOC00100',
       'PDOC00670', 'PDOC50006'],
      dtype='object')
Order of the chain: Index(['PDOC00030', 'PDOC50196', 'PDOC50006', 'PDOC00100', 'PDOC00662',
       'PDOC00064', 'PDOC00343', 'PDOC50106', 'PDOC00653', 'PDOC00154',
       'PDOC00018', 'PDOC50007', 'PDOC00791', 'PDOC50002', 'PDOC50003',
       'PDOC00014', 'PDOC50156', 'PDOC00670', 'PDOC50001', 'PDOC50017',
       'PDOC50199', 'PDOC00660', 'PDOC00561', 'PDOC00380', 'PDOC00750',
       'PDOC00271', 'PDOC00224'],
      dtype='object')
Order of the chain: Index(['PDOC00670', 'PDOC00380', 'PDOC50003', 'PDOC50007', 'PDOC00064',
       'PDOC50196', 'PDOC00271', 'PD

/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


--- Data Information ---
dataset: /Volumes/Samsung_T5/research/data/small_datasets/genbase/
number of label: 27
number of attribute: 1185
number of instance: 662 


--- Order of the chain ---
Index(['PDOC50007', 'PDOC00791', 'PDOC00100', 'PDOC00343', 'PDOC50196',
       'PDOC00662', 'PDOC50017', 'PDOC00064', 'PDOC00653', 'PDOC50002',
       'PDOC00750', 'PDOC00561', 'PDOC50199', 'PDOC00660', 'PDOC00154',
       'PDOC50006', 'PDOC50156', 'PDOC50106', 'PDOC50001', 'PDOC00030',
       'PDOC00224', 'PDOC00670', 'PDOC00018', 'PDOC00014', 'PDOC50003',
       'PDOC00271', 'PDOC00380'],
      dtype='object')

--- Performance ---
coverage_error = 1.27 ( avg_label_per_instance = 1.25 )
ranking_loss = 0.0
hamming_loss = 0.0
f1_macro = 0.57
f1_micro = 0.95
Jaccard_Index = 1.0
zero_one_error = 0.09

- f1 for each label -
label_PDOC00154 = 0.94
label_PDOC00343 = 1.0
label_PDOC00271 = 0.97
label_PDOC00064 = 1.0
label_PDOC00791 = 0.99
label_PDOC00380 = 0.94
label_PDOC50007 = 0.78
label_PDOC00224 = 1.0

## Ensemble CC using naive Bayes (E = 50)

In [43]:
# read data
data, label = read_data(dataPath)

# train - test
print("------ Binary Relevance using Naive Bayes ------")
ECC_test(data, label, dataPath, random_state=3071980, ensemble=50)

------ Binary Relevance using Naive Bayes ------
Order of the chain: Index(['PDOC00750', 'PDOC50003', 'PDOC50196', 'PDOC00030', 'PDOC00018',
       'PDOC50002', 'PDOC50106', 'PDOC50001', 'PDOC00271', 'PDOC50006',
       'PDOC00670', 'PDOC00561', 'PDOC00662', 'PDOC00660', 'PDOC00791',
       'PDOC00224', 'PDOC00154', 'PDOC00014', 'PDOC50156', 'PDOC50007',
       'PDOC50017', 'PDOC00064', 'PDOC00380', 'PDOC00100', 'PDOC00343',
       'PDOC50199', 'PDOC00653'],
      dtype='object')
Order of the chain: Index(['PDOC00014', 'PDOC00100', 'PDOC50002', 'PDOC00064', 'PDOC50003',
       'PDOC50199', 'PDOC00018', 'PDOC00662', 'PDOC00343', 'PDOC00380',
       'PDOC00791', 'PDOC00271', 'PDOC50017', 'PDOC50006', 'PDOC00660',
       'PDOC00561', 'PDOC00653', 'PDOC50001', 'PDOC50007', 'PDOC00670',
       'PDOC00154', 'PDOC50196', 'PDOC50156', 'PDOC50106', 'PDOC00224',
       'PDOC00750', 'PDOC00030'],
      dtype='object')
Order of the chain: Index(['PDOC50007', 'PDOC00791', 'PDOC00014', 'PDOC00154', 

Order of the chain: Index(['PDOC50003', 'PDOC00014', 'PDOC00750', 'PDOC00561', 'PDOC00380',
       'PDOC00660', 'PDOC00154', 'PDOC00791', 'PDOC50001', 'PDOC50196',
       'PDOC50006', 'PDOC00343', 'PDOC00018', 'PDOC50007', 'PDOC00224',
       'PDOC50106', 'PDOC00271', 'PDOC00662', 'PDOC50002', 'PDOC00030',
       'PDOC00064', 'PDOC50199', 'PDOC50156', 'PDOC00670', 'PDOC00653',
       'PDOC50017', 'PDOC00100'],
      dtype='object')
Order of the chain: Index(['PDOC00380', 'PDOC00018', 'PDOC00154', 'PDOC00224', 'PDOC00064',
       'PDOC00662', 'PDOC00653', 'PDOC50006', 'PDOC00791', 'PDOC00750',
       'PDOC50007', 'PDOC50017', 'PDOC00271', 'PDOC00561', 'PDOC50003',
       'PDOC00014', 'PDOC00030', 'PDOC50196', 'PDOC00343', 'PDOC00670',
       'PDOC50199', 'PDOC50156', 'PDOC50001', 'PDOC00660', 'PDOC50002',
       'PDOC50106', 'PDOC00100'],
      dtype='object')
Order of the chain: Index(['PDOC00670', 'PDOC50156', 'PDOC50007', 'PDOC50106', 'PDOC50001',
       'PDOC00224', 'PDOC00380', 'PD

Order of the chain: Index(['PDOC50003', 'PDOC00653', 'PDOC00662', 'PDOC00014', 'PDOC50017',
       'PDOC00750', 'PDOC50196', 'PDOC00064', 'PDOC00561', 'PDOC50156',
       'PDOC00660', 'PDOC50199', 'PDOC50002', 'PDOC00154', 'PDOC00670',
       'PDOC00271', 'PDOC50001', 'PDOC00100', 'PDOC00343', 'PDOC00380',
       'PDOC00791', 'PDOC50007', 'PDOC00224', 'PDOC00018', 'PDOC50006',
       'PDOC00030', 'PDOC50106'],
      dtype='object')
Order of the chain: Index(['PDOC00224', 'PDOC50106', 'PDOC50002', 'PDOC00653', 'PDOC50001',
       'PDOC00670', 'PDOC00662', 'PDOC50006', 'PDOC50007', 'PDOC00750',
       'PDOC00660', 'PDOC50156', 'PDOC00380', 'PDOC00271', 'PDOC00100',
       'PDOC00018', 'PDOC00064', 'PDOC00561', 'PDOC50003', 'PDOC50196',
       'PDOC00030', 'PDOC50199', 'PDOC00791', 'PDOC00014', 'PDOC50017',
       'PDOC00343', 'PDOC00154'],
      dtype='object')
Order of the chain: Index(['PDOC50196', 'PDOC00660', 'PDOC50003', 'PDOC00343', 'PDOC00064',
       'PDOC50106', 'PDOC00561', 'PD

/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [44]:
# read data
data, label = read_data(dataPath)

ensemble=50

# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

X_train = data[data.index.isin(index)==False]
X_test = data[data.index.isin(index)==True]
y_train = label[label.index.isin(index)==False]
y_test = label[label.index.isin(index)==True]

print("-- test index --")
print(X_test.index)

# ensemble
y_pred_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
y_prob_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
for i in range(ensemble):
    # training
    #print("--- start training ---\n")
    classifier_list, training_time, order = naiveBayes_multi_label_training(X_train, y_train)

    # print orders
    print("Order of the chain:",label.columns[order])

    # testing
    #print("--- start testing ---\n")
    y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order)

    y_predict.columns = label.columns[order]
    y_prob.columns = label.columns[order]
    y_predict = y_predict[label.columns]
    y_prob = y_prob[label.columns]

    y_pred_ensemble = y_pred_ensemble + y_predict
    y_prob_ensemble = y_prob_ensemble + y_prob

y_pred_ensemble = (((y_pred_ensemble / ensemble) >= 0.5)*1).astype('int')
y_prob_ensemble = y_prob_ensemble / ensemble 

# evaluation
performance = evaluation(y_pred_ensemble, y_prob_ensemble, y_test)


# print data information
print("--- Data Information ---")
print("dataset:", dataPath)
print("number of label:",n_label)
print("number of attribute:",n_attr)
print("number of instance:",n_instance,"\n")

# print orders
print("\n--- Order of the chain ---")
print(label.columns[order])
print("")

# get confusion matrix
get_confusion_matrix(y_pred_ensemble, y_test, y_test.columns)

# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

-- test index --
Int64Index([  5,   7,   8,  15,  16,  17,  21,  23,  24,  31,
            ...
            648, 650, 651, 652, 655, 656, 657, 658, 660, 661],
           dtype='int64', length=331)
Order of the chain: Index(['PDOC00100', 'PDOC50199', 'PDOC00014', 'PDOC00670', 'PDOC50196',
       'PDOC50001', 'PDOC00271', 'PDOC00660', 'PDOC00018', 'PDOC50002',
       'PDOC00224', 'PDOC50003', 'PDOC00791', 'PDOC50156', 'PDOC00750',
       'PDOC50017', 'PDOC00343', 'PDOC50006', 'PDOC50007', 'PDOC00662',
       'PDOC00154', 'PDOC00653', 'PDOC00030', 'PDOC00064', 'PDOC00380',
       'PDOC50106', 'PDOC00561'],
      dtype='object')


/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


Order of the chain: Index(['PDOC50196', 'PDOC00380', 'PDOC50199', 'PDOC50006', 'PDOC00064',
       'PDOC00030', 'PDOC50007', 'PDOC50001', 'PDOC00670', 'PDOC00750',
       'PDOC50156', 'PDOC00561', 'PDOC00224', 'PDOC00271', 'PDOC50003',
       'PDOC00154', 'PDOC00343', 'PDOC50017', 'PDOC00660', 'PDOC50002',
       'PDOC00018', 'PDOC00662', 'PDOC00100', 'PDOC00653', 'PDOC00014',
       'PDOC50106', 'PDOC00791'],
      dtype='object')
Order of the chain: Index(['PDOC00030', 'PDOC50001', 'PDOC50156', 'PDOC00100', 'PDOC50003',
       'PDOC00660', 'PDOC00343', 'PDOC00018', 'PDOC00224', 'PDOC00154',
       'PDOC00791', 'PDOC00653', 'PDOC50017', 'PDOC50199', 'PDOC50106',
       'PDOC00380', 'PDOC50196', 'PDOC50006', 'PDOC00670', 'PDOC00064',
       'PDOC50002', 'PDOC00662', 'PDOC00561', 'PDOC50007', 'PDOC00750',
       'PDOC00014', 'PDOC00271'],
      dtype='object')
Order of the chain: Index(['PDOC50001', 'PDOC50196', 'PDOC00380', 'PDOC50007', 'PDOC00660',
       'PDOC00018', 'PDOC00100', 'PD

Order of the chain: Index(['PDOC00670', 'PDOC50006', 'PDOC00018', 'PDOC00030', 'PDOC00064',
       'PDOC00653', 'PDOC50007', 'PDOC00662', 'PDOC50156', 'PDOC00791',
       'PDOC50003', 'PDOC50199', 'PDOC50001', 'PDOC50017', 'PDOC00750',
       'PDOC00343', 'PDOC50002', 'PDOC00660', 'PDOC00014', 'PDOC00100',
       'PDOC50106', 'PDOC00271', 'PDOC00380', 'PDOC00154', 'PDOC00561',
       'PDOC50196', 'PDOC00224'],
      dtype='object')
Order of the chain: Index(['PDOC50006', 'PDOC00653', 'PDOC00662', 'PDOC50003', 'PDOC00030',
       'PDOC00018', 'PDOC50106', 'PDOC50001', 'PDOC50199', 'PDOC00670',
       'PDOC00224', 'PDOC50196', 'PDOC50002', 'PDOC00343', 'PDOC00271',
       'PDOC50007', 'PDOC00380', 'PDOC00014', 'PDOC00750', 'PDOC00791',
       'PDOC50017', 'PDOC00064', 'PDOC00154', 'PDOC00561', 'PDOC50156',
       'PDOC00660', 'PDOC00100'],
      dtype='object')
Order of the chain: Index(['PDOC00343', 'PDOC00064', 'PDOC00750', 'PDOC00653', 'PDOC50199',
       'PDOC00014', 'PDOC50156', 'PD

Order of the chain: Index(['PDOC00561', 'PDOC50156', 'PDOC50006', 'PDOC00154', 'PDOC50106',
       'PDOC50196', 'PDOC00670', 'PDOC00018', 'PDOC00030', 'PDOC00750',
       'PDOC00380', 'PDOC00791', 'PDOC50017', 'PDOC50003', 'PDOC00653',
       'PDOC00064', 'PDOC50199', 'PDOC50007', 'PDOC00662', 'PDOC00014',
       'PDOC00271', 'PDOC00343', 'PDOC00224', 'PDOC50002', 'PDOC00660',
       'PDOC00100', 'PDOC50001'],
      dtype='object')
Order of the chain: Index(['PDOC00653', 'PDOC00030', 'PDOC00100', 'PDOC50003', 'PDOC00154',
       'PDOC50007', 'PDOC50199', 'PDOC00750', 'PDOC50006', 'PDOC50196',
       'PDOC00662', 'PDOC00380', 'PDOC00561', 'PDOC00018', 'PDOC50017',
       'PDOC50106', 'PDOC00343', 'PDOC00271', 'PDOC00014', 'PDOC00064',
       'PDOC00224', 'PDOC00791', 'PDOC50001', 'PDOC50002', 'PDOC50156',
       'PDOC00660', 'PDOC00670'],
      dtype='object')
Order of the chain: Index(['PDOC50001', 'PDOC50199', 'PDOC00100', 'PDOC00014', 'PDOC00380',
       'PDOC50007', 'PDOC00271', 'PD

/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## CC using ESKDB


In [45]:
# prepare data
def csv_to_arff(X, label_i, savePath):
    # get attributes
    attributes=[(X.columns[i],['0', '1']) for i in range(len(X.columns))]
    attributes.append(('label_'+label_i.name,['0', '1']))

    data=[]
    i = 0
    while i < len(label_i):
        attr_data = [str(j) for j in list(X.iloc[i,:])]
        label_data = [str(label_i[i])]
        row_data = attr_data+label_data
        data.append(row_data) 
        i+=1
    # set obj
    obj = {
       'description': u'',
       'relation': 'relation',
       'attributes': attributes,
       'data': data,
    }
    arff_data = arff.dumps(obj)
    w_file = open(savePath+label_i.name+".arff", "w")
    w_file.write(arff_data)
    w_file.close()

def get_arff(X, label, savePath):
    
    n_label = label.shape[1]
    # get orders
    order = random.sample(list(range(n_label)),n_label) # get orders
    
    #  get all arff files, one for each label
    for i in range(n_label):
        label_i = label.iloc[:,order[i]]
        print("--Running label:",label_i.name)
        csv_to_arff(X, label_i, savePath)
        
        label_i.name = 'label_' + label_i.name
        X = pd.concat([X, label_i], axis=1)
    print("--finished getting arff files")
    return order

def run_eskdb(label_arff, resultFile, k, l, e, i):
    command = "./run_ECC.sh "+resultFile+" "+k+" "+i+" "+l+" "+e+" "+label_arff
    subprocess.call("cd /Volumes/Samsung_T5/research/programme/research_python/", shell=True)
    #print(command)
    return subprocess.call(command, shell=True)

def get_result(resultPath):

    y_pred = pd.DataFrame()
    y_true = pd.DataFrame()
    y_prob = pd.DataFrame()
    names = []
    for file in os.listdir(resultPath):
        with open(os.path.join(resultPath,file), 'r') as f:
            try:
                lines = f.readlines()
            except:
                print(file)
            else:
                names.append(file[:-4])
                pred = []
                true = []
                prob = []
                train_time_total = 0
                test_time_total = 0
                error_marco = 0
                for line in lines:
                    if line.startswith('pred'):
                        pred.append(int(re.search('pred :\t(.)',line).group(1)))
                        true.append(int(re.search('true :\t(.)',line).group(1)))
                        prob.append(float(re.search('prob :\t(.*)',line).group(1)))
                    elif line.startswith('RSME'):
                        rsme = float(re.search('RSME :\t\t(.*)',line).group(1))
                    elif line.startswith('Error'):
                        error = float(re.search('Error :\t\t(.*)',line).group(1))
                    elif line.startswith("Training time"):
                        train_time = float(re.search('Training time :\s{1,}(.*)',line).group(1))
                        train_time_total = train_time_total + train_time
                    elif line.startswith("Testing time"):
                        test_time = float(re.search('Testing time :\s{1,}(.*)',line).group(1))
                        test_time_total = test_time_total + test_time
                    elif line.startswith("["):
                        para = line
                    elif line.startswith("test0Indexes"):
                        index = list(map(int,re.search('test0Indexes: {(.*)}',line).group(1).split(', ')))

                y_pred = pd.concat([y_pred,pd.DataFrame(pred)],axis=1)
                y_true = pd.concat([y_true,pd.DataFrame(true)],axis=1)
                y_prob = pd.concat([y_prob,pd.DataFrame(prob)],axis=1)
    y_pred.columns = names
    y_true.columns = names
    y_prob.columns = names
    print(para)
    print("number of label:", y_pred.shape[1])
    print("training time:",train_time_total)
    print("testing time:",test_time_total)
    return y_pred,y_true,y_prob,index

In [46]:
# read data
data, label = read_data(dataPath)
n_label = label.shape[1]

resultFile = dataset+'_k5_e20_i5000_CC'

# train and test on the first label
savePath = "/Users/jiangjunhao/Desktop/test/"+dataset+'/'

# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

# get orders
order = random.sample(list(range(n_label)),n_label) # get orders


## prepare data, get the arff file
for i in range(n_label):
    label_i = label.iloc[:,order[i]]
    #print("--Running label:",label_i.name)
    csv_to_arff(data, label_i, savePath)

    # run eskdb
    label_arff = os.path.join(savePath,label_i.name+'.arff')
    k = '5'
    i = '5000'
    l = '2'
    e = '20'

    run_eskdb(label_arff, resultFile, k, l, e, i)

    result = os.path.join("/Volumes/Samsung_T5/research/programme/ESKDB_HDP/result",resultFile, label_i.name+'.txt')
    with open(result, 'r') as f:
        try:
            lines = f.readlines()
        except:
            print(file)
        else:
            pred = []
            for line in lines:
                if line.startswith('pred'):
                    pred.append(int(re.search('pred :\t(.)',line).group(1)))
                elif line.startswith("test0Indexes"):
                    index = list(map(int,re.search('test0Indexes: {(.*)}',line).group(1).split(', ')))

            label.loc[index,label_i.name] = pred
            temp = label.loc[:,label_i.name]
            temp.name = 'label_'+label_i.name
            data = pd.concat([data, label.loc[:,label_i.name]],axis=1)

# get result
resultPath = '/Volumes/Samsung_T5/research/programme/ESKDB_HDP/result/'+resultFile+'/'
y_pred,y_true,y_prob,index = get_result(resultPath)
performance = evaluation(y_pred=y_pred, y_true=y_true, y_prob=y_prob)

# print orders:
print(label.columns[order])

# get confusion matrix
print("\n--- Confusion matrix ---")
get_confusion_matrix(y_pred, y_true, y_pred.columns)

# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

[-t, /Users/jiangjunhao/Desktop/test/genbase/PDOC50156.arff, -S, ESKDB, -K, 5, -I, 5000, -L, 2, -E, 20, -V, -M]

number of label: 27
training time: 6116.0
testing time: 299.0
Index(['PDOC00100', 'PDOC00653', 'PDOC00014', 'PDOC50199', 'PDOC50017',
       'PDOC00791', 'PDOC00343', 'PDOC00271', 'PDOC00670', 'PDOC50006',
       'PDOC50001', 'PDOC00018', 'PDOC00380', 'PDOC50007', 'PDOC00662',
       'PDOC50106', 'PDOC00660', 'PDOC00064', 'PDOC00224', 'PDOC50196',
       'PDOC50003', 'PDOC00154', 'PDOC00561', 'PDOC50002', 'PDOC00030',
       'PDOC00750', 'PDOC50156'],
      dtype='object')

--- Confusion matrix ---
--- Performance ---
coverage_error = 1.17 ( avg_label_per_instance = 1.25 )
ranking_loss = 0.0
hamming_loss = 0.0
f1_macro = 0.73
f1_micro = 0.99
Jaccard_Index = 1.0
zero_one_error = 0.03

- f1 for each label -
label_PDOC00154 = 0.0
label_PDOC00343 = 0.0
label_PDOC00271 = 0.0
label_PDOC00064 = 0.0
label_PDOC00791 = 1.0
label_PDOC00380 = 1.0
label_PDOC50007 = 1.0
label_PDOC00224 = 

/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## Ensemble CC using ESKDB(E=10)

In [59]:
# read data
data, label = read_data(dataPath)
n_label = label.shape[1]

# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

# ensemble
ensemble = 10

y_pred_ensemble = pd.DataFrame(np.zeros((int(label.shape[0]/2),label.shape[1])),columns=label.columns)
y_prob_ensemble = pd.DataFrame(np.zeros((int(label.shape[0]/2),label.shape[1])),columns=label.columns)
    
    
for i in range(ensemble):

    # read data
    data, label = read_data(dataPath)
    n_label = label.shape[1]

    # data set information
    n_label = label.shape[1]
    n_attr = data.shape[1]
    n_instance = data.shape[0]
    avg_label_per_instance = label.sum(axis=1).mean()

    # get orders
    order = random.sample(list(range(n_label)),n_label) # get orders

    # train and test on the first label
    savePath = "/Users/jiangjunhao/Desktop/test/"+dataset+'/'

    ## prepare data, get the arff file
    for i in range(n_label):
        label_i = label.iloc[:,order[i]]
        print("--Running label:",label_i.name)
        csv_to_arff(data, label_i, savePath)

        # run eskdb
        label_arff = os.path.join(savePath,label_i.name+'.arff')
        k = '5'
        i = '5000'
        l = '2'
        e = '20'

        run_eskdb(label_arff, resultFile, k, l, e, i)

        result = os.path.join("/Volumes/Samsung_T5/research/programme/ESKDB_HDP/result",resultFile, label_i.name+'.txt')
        with open(result, 'r') as f:
            try:
                lines = f.readlines()
            except:
                print(file)
            else:
                pred = []
                for line in lines:
                    if line.startswith('pred'):
                        pred.append(int(re.search('pred :\t(.)',line).group(1)))
                    elif line.startswith("test0Indexes"):
                        index = list(map(int,re.search('test0Indexes: {(.*)}',line).group(1).split(', ')))

                label.loc[index,label_i.name] = pred
                temp = label.loc[:,label_i.name]
                temp.name = 'label_'+label_i.name
                data = pd.concat([data, label.loc[:,label_i.name]],axis=1)

    # get result
    resultPath = '/Volumes/Samsung_T5/research/programme/ESKDB_HDP/result/'+resultFile+'/'
    y_pred,y_true,y_prob,index = get_result(resultPath)
    y_pred.columns = label.columns
    y_prob.columns = label.columns
    
    y_pred_ensemble = y_pred_ensemble + y_pred
    y_prob_ensemble = y_prob_ensemble + y_prob
    
    
y_pred_ensemble = (((y_pred_ensemble / ensemble) >= 0.5)*1).astype('int')
y_prob_ensemble = y_prob_ensemble / ensemble 

--Running label: PDOC50196
--Running label: PDOC50003
--Running label: PDOC50156
--Running label: PDOC00660
--Running label: PDOC00064
--Running label: PDOC00154
--Running label: PDOC50007
--Running label: PDOC50017
--Running label: PDOC50006
--Running label: PDOC50199
--Running label: PDOC00561
--Running label: PDOC00271
--Running label: PDOC00750
--Running label: PDOC00014
--Running label: PDOC50001
--Running label: PDOC00653
--Running label: PDOC00343
--Running label: PDOC00018
--Running label: PDOC50106
--Running label: PDOC00791
--Running label: PDOC00224
--Running label: PDOC00380
--Running label: PDOC00100
--Running label: PDOC00030
--Running label: PDOC00662
--Running label: PDOC50002
--Running label: PDOC00670
[-t, /Users/jiangjunhao/Desktop/test/genbase/PDOC50156.arff, -S, ESKDB, -K, 5, -I, 5000, -L, 2, -E, 20, -V, -M]

number of label: 27
training time: 5479.0
testing time: 110.0
--Running label: PDOC00030
--Running label: PDOC00343
--Running label: PDOC00670
--Running label

--Running label: PDOC00014
--Running label: PDOC50006
--Running label: PDOC50017
--Running label: PDOC00064
--Running label: PDOC50196
--Running label: PDOC00653
--Running label: PDOC50199
--Running label: PDOC00791
--Running label: PDOC00662
--Running label: PDOC00561
--Running label: PDOC50156
--Running label: PDOC00660
--Running label: PDOC00030
--Running label: PDOC50003
--Running label: PDOC50001
--Running label: PDOC50002
--Running label: PDOC00154
--Running label: PDOC00343
--Running label: PDOC00224
--Running label: PDOC50106
--Running label: PDOC50007
--Running label: PDOC00750
--Running label: PDOC00271
--Running label: PDOC00380
[-t, /Users/jiangjunhao/Desktop/test/genbase/PDOC50156.arff, -S, ESKDB, -K, 5, -I, 5000, -L, 2, -E, 20, -V, -M]

number of label: 27
training time: 6083.0
testing time: 499.0


In [60]:
performance = evaluation(y_pred=y_pred_ensemble, y_true=y_true, y_prob=y_prob_ensemble)

# print orders:
print(label.columns[order])

# get confusion matrix
print("\n--- Confusion matrix ---")
get_confusion_matrix(y_pred, y_true, y_pred.columns)

# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

Index(['PDOC00670', 'PDOC00018', 'PDOC00100', 'PDOC00014', 'PDOC50006',
       'PDOC50017', 'PDOC00064', 'PDOC50196', 'PDOC00653', 'PDOC50199',
       'PDOC00791', 'PDOC00662', 'PDOC00561', 'PDOC50156', 'PDOC00660',
       'PDOC00030', 'PDOC50003', 'PDOC50001', 'PDOC50002', 'PDOC00154',
       'PDOC00343', 'PDOC00224', 'PDOC50106', 'PDOC50007', 'PDOC00750',
       'PDOC00271', 'PDOC00380'],
      dtype='object')

--- Confusion matrix ---


/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


--- Performance ---
coverage_error = 1.18 ( avg_label_per_instance = 1.25 )
ranking_loss = 0.0
hamming_loss = 0.0
f1_macro = 0.75
f1_micro = 0.99
Jaccard_Index = 1.0
zero_one_error = 0.03

- f1 for each label -
label_PDOC00154 = 0.0
label_PDOC00343 = 0.0
label_PDOC00271 = 0.0
label_PDOC00064 = 0.0
label_PDOC00791 = 1.0
label_PDOC00380 = 1.0
label_PDOC50007 = 1.0
label_PDOC00224 = 1.0
label_PDOC00100 = 1.0
label_PDOC00670 = 0.67
label_PDOC50002 = 1.0
label_PDOC50106 = 1.0
label_PDOC00561 = 1.0
label_PDOC50017 = 0.73
label_PDOC50003 = 1.0
label_PDOC50006 = 1.0
label_PDOC50156 = 0.0
label_PDOC00662 = 1.0
label_PDOC00018 = 1.0
label_PDOC50001 = 0.0
label_PDOC00014 = 1.0
label_PDOC00750 = 1.0
label_PDOC50196 = 1.0
label_PDOC50199 = 0.96
label_PDOC00660 = 1.0
label_PDOC00653 = 1.0
label_PDOC00030 = 1.0


# Result

recall 体现了分类模型H对正样本的识别能力，recall 越高，说明模型对正样本的识别能力越强.

precision 体现了模型对负样本的区分能力，precision越高，说明模型对负样本的区分能力越强。F1-score 是两者的综合。F1-score 越高，说明分类模型越稳健。